In [ ]:
# NB: This code can only be used if you have the required dataset, which is not public.
# Thus, the original dataset is not included.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

# NB: user needs to install xlrd

In [ ]:
from tf.app import use
A = use("etcbc/bhsa", hoist=globals())

In [ ]:
verbs_dataset = pd.read_excel("verbs_dataset.xls")

In [ ]:
# print the verb root info column
verbs_dataset.info_verbroot

In [ ]:
# extract all the different verbs
verbs_dataset.info_verbroot.unique()

In [ ]:
# show the length of the list of verbs
verbs_dataset.info_verbroot.nunique()

In [ ]:
verbs_dataset.info_verbroot.value_counts()

In [ ]:
# save to a csv file
verbs_dataset.info_verbroot.value_counts().to_csv("verbs_counts.csv", sep=";")

In [ ]:
# show the file
pd.read_csv("verbs_counts.csv", sep=";")

In [ ]:
# bcv stands for book, chapter, verse

bcv_verbs = verbs_dataset.groupby("info_verbroot").first().info_refer.values
info_verbroot = verbs_dataset.groupby("info_verbroot").first().index
#bcv_verbs

In [ ]:
# How to retrieve book, chapter, name for each verb

# retrieve a section book chapter verse from the column refer in the dataset
book_names = {
    '1Ch':'1_Chronicles',
    '1Kgs':'1_Kings',
    '1Sam':'1_Samuel',
    '2Ch':'2_Chronicles',
    '2Kgs':'2_Kings',
    '2Sam':'2_Samuel',
    'Amos':'Amos',
    'Dan':'Daniel',
    'Deut':'Deuteronomy',
    'Qoh':'Ecclesiastes', 
    'Est':'Esther',
    'Ex':'Exodus',
    'Ezek':'Ezekiel',
    'Ezra':'Ezra',
    'Gen':'Genesis',
    'Hab':'Habakkuk', # absent from the dataset
    'Hag':'Haggai',
    'Hosea':'Hosea', # absent from the dataset
    'Isa':'Isaiah',
    'Jer':'Jeremiah',
    'Job':'Job',
    'Joel':'Joel',
    'Jonah':'Jonah',
    'Josh':'Joshua',
    'Jud':'Judges',
    'Lam':'Lamentations', # absent from the dataset
    'Lev':'Leviticus',
    'Mal':'Malachi',
    'Micah':'Micah', # absent from the dataset
    'Nahum':'Nahum', # absent from the dataset
    'Neh':'Nehemiah',
    'Num':'Numbers',
    'Obad':'Obadiah', # absent from the dataset
    'Prov':'Proverbs', # absent from the dataset
    'Ps':'Psalms',
    'Rut':'Ruth',
    'Song_of_songs':'Song_of_songs', # absent from the dataset
    'Zech':'Zechariah',
    'Zephaniah':'Zephaniah', # absent from the dataset
}
# sections contains groups of book chapter verse

sections = []

# separate book from chapter:verse and add the data to the sections list
for item in bcv_verbs:
    sections.append(item.split(sep=" "))

# how to split the data into a usable section book, chapter, verse
for item in sections:
    # split chapter from verse
    item[1:] = item[1].split(sep=":")
    
    # transform chapter:verse to integers
    item[1] = int(item[1])
    item[2] = int(item[2])
    
    # retrieve the ETCBC book name from the dictionary and replace it in sections
    item[0] = book_names[item[0]]
    
print(sections)

# define a function to retrieve the lexeme fronm a section book, chapter, verse
def verse_lex(section):
    verse = (T.nodeFromSection(section))
    return " ".join([F.lex.v(w) for w in L.d(verse, "word")])

verses_lexemes = []

for section in sections:
    verses_lexemes.append(verse_lex(section))

In [ ]:
# list(zip(info_verbroot, sections, verses_lexemes))
pd.DataFrame({"info_verbroot": info_verbroot, "bhsa_lex": [""]*len(info_verbroot), "sections": sections, "verses_lexemes": verses_lexemes}).to_csv("info_verbroot_with_lexemes.csv", sep=";", index=False)

In [ ]:
# try out the verse_lex function
verse_lex(['Isaiah', 6, 6])

In [ ]:
df = pd.read_csv("verb_occ_count_bhsa_lex.csv", sep=";")
df

# sort the verb_occurrences values in a descending order
df[["bhsa_lex", "verb_occurrences"]].sort_values("verb_occurrences", ascending=False).to_csv("verb_count_etcbc_lex.csv", sep=";", index=False)

In [ ]:
# create a dictionary with the verbs transcriptions from original dataset linked to the BHSA lexexmes
lexemes_dict = dict(zip(df.info_verbroot, df.bhsa_lex))
lexemes_dict

In [ ]:
# add a nex column to the dataset with the BHSA lexemes
verbs_dataset.insert(1, "bhsa_lex", verbs_dataset.info_verbroot.map(lambda x: lexemes_dict[x]))

In [ ]:
# replace the column info_verbroot
info_verbroot = verbs_dataset.pop("info_verbroot")
verbs_dataset.insert(2, "info_verbroot", info_verbroot)
verbs_dataset.to_csv("verbs_dataset_modified.csv", sep=";")

In [ ]:
# count the values in the book column
verbs_dataset.book.value_counts()

In [ ]:
verbs_dataset.gc_prep.value_counts()

In [ ]:
verbs_dataset.gc2.value_counts()

In [ ]:
pd.crosstab(verbs_dataset.bhsa_lex, verbs_dataset.gc_prep).to_csv("verbs_and_prep.csv", sep=";")

In [ ]:
pd.crosstab(verbs_dataset.bhsa_lex, verbs_dataset.gc2).to_csv("verbs_and_hey.csv", sep=";")